In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 32
p1 = .5

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        
    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(self.convtrans3(x), scale_factor=scale_factor)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             320
       BatchNorm2d-2           [-1, 32, 32, 32]              64
            Conv2d-3           [-1, 64, 16, 16]          18,496
       BatchNorm2d-4           [-1, 64, 16, 16]             128
   ConvTranspose2d-5             [-1, 32, 8, 8]          18,464
       BatchNorm2d-6             [-1, 32, 8, 8]              64
   ConvTranspose2d-7           [-1, 16, 16, 16]           4,624
       BatchNorm2d-8           [-1, 16, 16, 16]              32
   ConvTranspose2d-9            [-1, 2, 16, 16]             290
Total params: 42,482
Trainable params: 42,482
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.85
Params size (MB): 0.16
Estimated Total Size (MB): 1.01
---------------------------------------------

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00375270 (0.04170848), PSNR 24.25656319 (19.65513851), SSIM 0.63152838 (0.38256687)
Finished training epoch 0
Validate: MSE 0.00395957 (0.00373437), PSNR 24.02352142 (24.36387218), SSIM 0.58832973 (0.64984987)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00320916 (0.00307403), PSNR 24.93608665 (25.15301156), SSIM 0.69967723 (0.71158124)
Finished training epoch 1
Validate: MSE 0.00344796 (0.00286440), PSNR 24.62437630 (25.49489607), SSIM 0.67379427 (0.73988462)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00253408 (0.00275294), PSNR 25.96180153 (25.62624294), SSIM 0.75579989 (0.75918210)
Finished training epoch 2
Validate: MSE 0.00309369 (0.00269877), PSNR 25.09522629 (25.76567089), SSIM 0.69834065 (0.76816856)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00254660 (0.00271129), PSNR 25.94038582 (25.69029037), SSIM 0.75437415 (0.76684354)
Finished training epoch 3
Validate: MSE 0.00301659 (0.0

Epoch: 30, MSE 0.00238292 (0.00244499), PSNR 26.22889709 (26.13849185), SSIM 0.77823925 (0.77113470)
Finished training epoch 30
Validate: MSE 0.00289059 (0.00250373), PSNR 25.39013100 (26.08174076), SSIM 0.70247364 (0.76833730)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00255537 (0.00245399), PSNR 25.92545891 (26.12567146), SSIM 0.75347602 (0.77067489)
Finished training epoch 31
Validate: MSE 0.00299818 (0.00266157), PSNR 25.23141861 (25.82953472), SSIM 0.70660889 (0.76742732)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00244288 (0.00244643), PSNR 26.12097931 (26.13905349), SSIM 0.77503139 (0.77039720)
Finished training epoch 32
Validate: MSE 0.00297306 (0.00248784), PSNR 25.26795578 (26.11116525), SSIM 0.70964921 (0.77441879)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00236702 (0.00244336), PSNR 26.25798607 (26.14255039), SSIM 0.77796489 (0.77078105)
Finished training epoch 33
Validate: MSE 0.00271500 (0.00242671), PSNR 

Epoch: 60, MSE 0.00192247 (0.00241217), PSNR 27.16139603 (26.19801171), SSIM 0.77745026 (0.77052588)
Finished training epoch 60
Validate: MSE 0.00274014 (0.00240397), PSNR 25.62226295 (26.25217721), SSIM 0.70351982 (0.76989005)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00235648 (0.00241171), PSNR 26.27736664 (26.19614601), SSIM 0.77739060 (0.77069396)
Finished training epoch 61
Validate: MSE 0.00269955 (0.00279436), PSNR 25.68709183 (25.65611710), SSIM 0.70816720 (0.76364709)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00268972 (0.00242442), PSNR 25.70292664 (26.17569864), SSIM 0.78669012 (0.77011474)
Finished training epoch 62
Validate: MSE 0.00292019 (0.00266333), PSNR 25.34588051 (25.81878537), SSIM 0.70420545 (0.76316283)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00284047 (0.00240831), PSNR 25.46609879 (26.20722487), SSIM 0.76911539 (0.77061237)
Finished training epoch 63
Validate: MSE 0.00284088 (0.00240690), PSNR 

Epoch: 90, MSE 0.00218971 (0.00240928), PSNR 26.59612465 (26.20351879), SSIM 0.76274079 (0.77022549)
Finished training epoch 90
Validate: MSE 0.00290069 (0.00242299), PSNR 25.37498474 (26.22786599), SSIM 0.70580447 (0.77212294)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00233628 (0.00240270), PSNR 26.31475067 (26.21655249), SSIM 0.78332955 (0.77026584)
Finished training epoch 91
Validate: MSE 0.00281242 (0.00240804), PSNR 25.50919342 (26.25119293), SSIM 0.70886534 (0.77035299)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00227369 (0.00239934), PSNR 26.43268204 (26.22257239), SSIM 0.77487916 (0.77032332)
Finished training epoch 92
Validate: MSE 0.00264828 (0.00272261), PSNR 25.77035141 (25.75272231), SSIM 0.69856429 (0.75785940)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00236515 (0.00241039), PSNR 26.26142120 (26.20112256), SSIM 0.77689761 (0.77039447)
Finished training epoch 93
Validate: MSE 0.00279033 (0.00323671), PSNR 

Epoch: 120, MSE 0.00215827 (0.00239525), PSNR 26.65894508 (26.22763919), SSIM 0.79350960 (0.77049778)
Finished training epoch 120
Validate: MSE 0.00276495 (0.00260749), PSNR 25.58313179 (25.92913628), SSIM 0.70776796 (0.76536020)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00186694 (0.00239428), PSNR 27.28868484 (26.23066637), SSIM 0.78110904 (0.77060328)
Finished training epoch 121
Validate: MSE 0.00292810 (0.00238764), PSNR 25.33413696 (26.28675850), SSIM 0.70581710 (0.77188678)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00283162 (0.00240111), PSNR 25.47965622 (26.22134896), SSIM 0.74596339 (0.77018920)
Finished training epoch 122
Validate: MSE 0.00289311 (0.00243177), PSNR 25.38634491 (26.20922177), SSIM 0.71028197 (0.77090445)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00285029 (0.00243561), PSNR 25.45110893 (26.20293750), SSIM 0.71237963 (0.77085616)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()